# 3. ROADSENSE introduction

Given the basics of point cloud simulation from scratch with python, let's move to the last part of the 1st block, which consists in a brief introduction to the **ROADSENSE simulator**: https://github.com/GeoTechUVigo/ROADSENSE

<center><img src="images/roadsense_github.png" alt="Drawing" style="width: 800px;"/></center>

**ROADSENSE** is a simulation software that allows the generation of synthetic point clouds datasets within **different transportation systems and forestry scenarios**. It was published alongside a case study in https://doi.org/10.3390/infrastructures9030058, where different experiments were carried with a DL model, PointNet++ (which will be used in the next section of this course) and compared with the results retrieved when departing from synthetic data from other state-of-the-art simulator HELIOS++.

As stated before, ROADSENSE allows the generation of synthetic point clouds in both forested regions as well as places partially covered with vegetation where different types of roads coexist.

<center><img src="images/nubes_sinteticas.png" alt="Drawing" style="width: 800px;"/></center>


The methodology followed by ROADSENSE during the generation protocols is quite similar to the one presented in the previous section. For example, in case of forest scenarios without road presence, the process can be sorted as follows: **generation of the Digital Terrain Model (DTM)**, **generation of modified tree segments** and, finally, **merge of the semantic information in a single point cloud**. However, the case of transportation scenes requires the implementation of new 3D designs concerning human-made elements, like barriers, traffic signals or even natural elements like slopes and their surrounding vegetation.


<center><img src="images/autopista_sintetica_1.png" alt="Drawing" style="width: 800px;"/></center>


## 3.1. General filesystem structure

- organization and explanation of what do each script

```
└── ROADSENSE/
    ├── roadsense.py
    ├── README.md
    ├── config/
    │   └── forest_road_backbone.txt
    ├── data/
    │   └── trees_pcd/
    │       ├── arbol_0.pcd
    │       ├── arbol_1.pcd
    │       ├── (...)
    │       ├── arbol_117.pcd
    │       └── arbol_118.pcd
    ├── output/
    │   └── README.md
    └── utils/
        ├── aei.jpeg
        ├── cross_section.py
        ├── DTM_road_wizard.py
        ├── reading_trees.py
        ├── road_generator.py
        ├── signal_generator.py
        ├── tree_wizard.py
        └── INTERSECCION_MESH_TRAYECTORIA.py
```

Here a brief explanation of the main characteristics of the filesystem tree:

**roadsense.py:** Main script of the simulator

**/data/trees_pcd/:** Folder where the TLS tree models are stored

**/output/:** Folder where the resulting datasets are stored

**/utils/cross_section.py:** Script with the cross section dependencies

**/utils/DTM_road_wizard.py:** Script for designing the geometry of the full scene except vegetation

**/utils/reading_trees.py:** Script for reading the TLS tree models stored in the /data folder

**/utils/road_generator.py:** Script for designing the road-related geometries

**/utils/signal_generator.py:** Script with the different traffic signals modelization protocols

**/utils/tree_wizard.py:** Script with the euclidean transformations criteria for tree data augmentation

**/utils/INTERSECCION_MESH_TRAYECTORIA.py:** Script for simulating a scanner trajectory (required for spectral simulations)

**/config/forest_road_backbone.txt:** Contains all of the parameters required to simulate the point clouds, considering all different scenarios and settings, and it is written in a .txt format, which can be easily modified using a text editor. All parameters defined in the config file are summarized as follows:

    - seed (index 0): This is a random seed used in intermediate randomizers. Options: “None” or “integer number”. If the None option is selected, the simulator will consider the time.time() value. Specifying an int number will make all clouds and their features equal.
    
    - road (index 1): This indicates whether the resulting point clouds will have a road or not. Options: “True” or “False”.
    
    - spectral_mode (index 2): This parameter is used in the case of performing a spectral simulation on the resulting point clouds. Options: “True” or “False”. If True is set, the simulator will take longer to generate each point cloud since there are several intermediate calculations, like the normals estimation or the virtual trajectory simulation.
    
    - road_type (index 3): This indicates the type of road to simulate. Options: “highway”, “national”, “local”, or “mixed”. If the “mixed” option is selected, the resulting point cloud dataset will consist of a mix of different types of roads.
    
    - tree_path (index 4): This represents the absolute path to the directory where all of the tree segments are stored.
    
    - number_of_clouds (index 5): This represents the total number of point clouds that will be simulated. It must be an integer number greater than zero.
    
    - scale (index 6): This is a geometric scale of the resulting point cloud. Bigger scales will result in bigger sized point clouds but with less point densities. It must be an integer number greater than 0.
    
    - number_of_trees (index 7): This represents the total number of trees per cloud. It must be an integer number greater than 0.
    
    - number_of_transformations (index 8): This is the number of Euclidean transformations that will suffer each tree segment. It must be an int number. If the total number of tree segments specified in the “tree_path” is less than the “number_of_trees”, then the simulator will randomly repeat all segments until completion.
    
    - X_buffer (indexes 9–11): This represents the width of the “X” element in meters. “X” can be “road”, “shoulder”, or “berm”. It must be a float number greater than 0.
    
    - slope_buffer (index 12): This represents the width of the slope in meters. Options: “float number greater than 0” or “random”. If “random” is set, then there will be slopes with different widths in the final point clouds.
    
    - noise_X (indexes 13–18): This represents the noise threshold in each XYZ direction per point of the “X” element. “X” can be “DTM”, “road”, “shoulder”, “slope”, “berm”, or “refugee_island”. It must be in the “(x,y,z)” bracket notation, where x, y, and z must be float numbers.
    
    - number_of_trees_refugee_island (index 19): This represents the number of trees in the median strip. It must be an integer number.
    
    - number_points_DTM (index 20): This represents the number of points in the edge of the DTM grid. It should be ~10 times the scale. It must be an integer number greater than 0.
    
    - vertical_pumping (index 21): This indicates whether the simulated road will have vertical pumping or not, i.e., if the points closer to the axis road will have different heights than the ones that are further. Options: “True” or “False”.

## 3.2. Running your first simulations

- Ensure to ```git clone``` the repository first!
- Play with different parameters setup